### Trading Testing

Testing delta probability for weeklies and monthlies
    - Does delta match probability of occurrence
    - Check Kelly criterion for long straddles/delta neutral back ratios
        - To calculate expected losses and gains, use options pricing module to interpolate prices at given percentage
        moves and use the normal pdf as the probability weightings
    
Check earnings returns post announcement factors
    - Factors:
        - Number of times beaten earnings (Dummy Variable)
        - Consecutive earnings beats
        - Consecutive earnings upsets
        - 3 Month Trend before earnings
        - YTD Trend
        - Momentum of monthly returns (20 day, 60 day)


In [1]:
import datetime as dt

import numpy as np
import pandas as pd
from pandas_datareader.data import Options
from py_vollib.black_scholes_merton.implied_volatility import *
# ts = TimeSeries(key='5HZEUI5AFJB06BUK',output_format='pandas')
import py_vollib
from py_vollib.black_scholes_merton.implied_volatility import *
from py_vollib.black_scholes_merton.greeks.analytical import *
import plotly
import os
import pandas_market_calendars as mcal
import json
from pandas.io.json import json_normalize
import urllib.request as req
import time
nyse = mcal.get_calendar('NYSE')
from helpers import *

from scipy.stats import norm as norm
# '''
# functions list:
    
#     maturities(dt.datetime()) --> [float(front_wgt), float(back_wgt)]
    
#     optionslam_scrape(str[ticker]) --> dict[earnings]
    
#     yahoo_table_parse(str[raw_html_table]) --> DataFrame[ticker]
    
#     yahoo_earnings(dt.datetime()) --> DataFrame[earnings_on_date]
    
#     fundamentals(str[ticker]) --> DataFrame[stock_fundamentals]
    
#     get_fundas(list[ticker_lst]) --> DataFrame[stock_fundamentals]
    
#     historical_data(str[ticker], int[day_number], int[rolling_window], outsize[str]) --> DataFrame[daily_stock_data]
    
#     current_volatility(list[ticker_lst], int[roll]) --> DataFrame[stock_volatilities]
    
#     all_options(str[ticker], bool[greeks]) --> DataFrame[options_chains]
    
#     earnings_condor(str[ticker], int[max_gap], int[dte_thresh], float[|money_thresh| <= 1]) -- DataFrame[condors], DataFrame[puts], DataFrame[calls]
    
#     write_excel(str[filename], list[str[sheetnames]], list[dataframes]) --> void()
    
#     curr_stock_data(str[ticker]) --> DataFrame[stock_info]
    
#     curr_batch_quotes(list_of_string[tickers]) --> DataFrame[stock_info]

#     past_earnings(str[ticker]) --> DataFrame[earnings_info]

#     earnings_history(str[ticker]) --> [DataFrame[earnings_estimate], DataFrame[past_earnings], DataFrame[earnings_estimate_change]]
    
#     av_data(str[ticker]) --> DataFrame[ticker_open, ticker_close]

#     av_batch(list_of_str[tickers]) --> DataFrame[tickers_closes]

#     check_mkt_corr(int[corr_rolling_window],int[plot_window]) --> DataFrame[rolling_corr]

#     vvix_check() --> DataFrame[VVIX Data]

#     earnings_longs(list_of_str[ticker], float[bid_ask_spread]) --> DataFrame[option_chains]

#     all_options_v2(str[ticker], int[dte_ub], int[dte_lb], float[moneyness]) --> DataFrame[option_chains]

#     yahoo_options_query(str[ticker], int[dte_ub], int[dte_lb]) --> DataFrame[option_chains]

#     greek_calc(DataFrame[option_chain], str[prem_price_use], str[day_format], float[interest_rate], float[dividend_rate])

#     price_sim(DataFrame[options_df], float[price_change], float[vol_change], int[days_change], str[output = 'All'],
#               str[skew = 'flat'], str[day_format = 'trading'], float[interest_rate = 0.0193], float[dividend_rate = 0],
#               float[prem_price_use = 'Mid'])


#     position_sim(DataFrame[position_df], list_of_int[holdings], int[shares],
#                  float[price_change], float[vol_change], int[dte_change], str[output = 'All'],
#                  str[skew = 'flat'], str[prem_price_use = 'Mid'], str[day_format = 'trading'], 
#                  float[interest_rate = 0.0193], float[dividend_rate = 0])

#     yahoo_fundamentals(list_of_str[tickers]) --> DataFrame[fundamentals]

# '''

In [2]:
# Pulling Indices Data

spx = pd.read_csv('spx.csv', index_col = 0)[['Open','Close']]
spx.columns = ['spx_{0}'.format(x) for x in spx.columns]

vix = pd.read_csv('vix.csv', index_col = 0)[['Open','Close']]
vix.columns = ['vix_{0}'.format(x) for x in vix.columns]

vvix = pd.read_csv('vvix.csv', index_col = 0)[['Close']]
vvix.columns = ['vvix']

yields = pd.read_csv('https://www.quandl.com/api/v3/datasets/USTREASURY/YIELD.csv?api_key=dzmzEExntfap7SNx5p6t', index_col = 0)
df = pd.concat([spx,yields[yields.columns[1:]]], axis = 1).dropna()
df.index = pd.to_datetime(df.index)
df[df.columns] = df[df.columns].apply(pd.to_numeric, errors='coerce')

In [3]:
# Creating full calendar

nyse_dates = nyse.schedule(start_date=df.index[0].date().strftime('%Y-%m-%d'), end_date=df.index[-1].date().strftime('%Y-%m-%d'))
date_idx = [x.date() for x in mcal.date_range(nyse_dates, frequency='1D')]

date_df = pd.DataFrame(columns = ['Trading_Day'],
                       index = range(len(date_idx)))
date_df.index = date_idx
date_df.index = pd.to_datetime(date_df.index)
date_df['Trading_Day'] = True

start_date = dt.datetime(1993,12,1).date()

full_cal = []
curr_date = start_date
while curr_date != dt.datetime.today().date():
    full_cal.append(curr_date)
    curr_date = curr_date + dt.timedelta(days = 1)
    
calendar_df = pd.DataFrame(columns = ['Weekday'],
                           index = full_cal)
calendar_df.index = pd.to_datetime(calendar_df.index)
calendar_df['Weekday'] = calendar_df.index.weekday
date_df = calendar_df.join(date_df).fillna(False)

In [4]:
spx_check = date_df.join(df)
spx_check = spx_check[spx_check['Weekday'].isin([0,1,2,3,4])]
spx_check['Week'] = spx_check.index.week
spx_check['Month'] = spx_check.index.month
spx_check['Year'] = spx_check.index.year
spx_check['Month_Expiry'] = False
spx_check['New_Month'] = False
spx_check['Week_Start'] = False
spx_check['Week_End'] = False

# Marking Monthly Expirations for SPX and Following Trading Day
for idx, row in spx_check[['Month','Year']].drop_duplicates().iterrows():
    curr_eigth_day = dt.date(row['Year'],row['Month'],7)
    curr_third_fri = curr_eigth_day - dt.timedelta(dt.date(row['Year'],row['Month'],3).weekday()) + dt.timedelta(14)
    curr_third_thurs = curr_eigth_day - dt.timedelta(dt.date(row['Year'],row['Month'],4).weekday()) + dt.timedelta(14)
    curr_third_wed = curr_eigth_day - dt.timedelta(dt.date(row['Year'],row['Month'],5).weekday()) + dt.timedelta(14)
    
    # Marking monthly expiry days: Third Friday of every month, otherwise third thursday
    if spx_check.loc[curr_third_fri,'spx_Open'] != np.nan:
        spx_check.loc[curr_third_fri,'Month_Expiry'] = True
        
    elif spx_check.loc[curr_third_thurs,'spx_Open'] != np.nan:
        spx_check.loc[curr_third_thurs,'Month_Expiry'] = True
        
    elif spx_check.loc[curr_third_wed,'spx_Open'] != np.nan:
        spx_check.loc[curr_third_wed,'Month_Expiry'] = True
        
# Marking Monthly Starting
for idx, row in spx_check[spx_check['Month_Expiry'] == True].iterrows():
    following_week = spx_check[spx_check.index > idx].head()
    for idx_2, row_2 in following_week.iterrows():
        if row_2['Trading_Day']:
            spx_check.loc[idx_2, 'New_Month'] = True
            break
            
# Setting Weekly Tags

for idx, row in spx_check[['Week','Year']].drop_duplicates().iterrows():
    current_week = spx_check[(spx_check['Week'] == row['Week']) &
                             (spx_check['Year'] == row['Year']) &
                             (spx_check['Trading_Day'] == True)]
    week_start = current_week.head(1)
    week_end = current_week.tail(1)
    
    try:
        spx_check.loc[week_start.index[0].date(), 'Week_Start'] = True
        spx_check.loc[week_end.index[0].date(), 'Week_End'] = True
    except:
        continue

spx_check['Monthlies'] = spx_check['Month_Expiry'] | spx_check['New_Month']
spx_monthly_check = spx_check[spx_check['Monthlies'] == True]

spx_check['Weeklies'] = spx_check['Week_Start'] | spx_check['Week_End']
spx_weekly_check = spx_check[(spx_check['Weeklies'] == True) &
                             (spx_check['Year'] > 1993) &
                             (spx_check['Year'] < 2018)]

# Filling in missing values
spx_monthly_check.spx_Open = spx_monthly_check.spx_Open.fillna((spx_monthly_check.spx_Open.shift() + 
                                                                spx_monthly_check.spx_Open.shift(-1))/2)
spx_monthly_check.spx_Close = spx_monthly_check.spx_Close.fillna((spx_monthly_check.spx_Close.shift() + 
                                                                  spx_monthly_check.spx_Close.shift(-1))/2)

spx_weekly_check.spx_Open = spx_weekly_check.spx_Open.fillna((spx_weekly_check.spx_Open.shift() + 
                                                              spx_weekly_check.spx_Open.shift(-1))/2)
spx_weekly_check.spx_Close = spx_weekly_check.spx_Close.fillna((spx_weekly_check.spx_Close.shift() + 
                                                                spx_weekly_check.spx_Close.shift(-1))/2)

C:\Users\Fang\Anaconda3\lib\site-packages\pandas\core\generic.py:2999: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [5]:
spx_weekly_check = spx_weekly_check[['spx_Open','spx_Close','Week_Start','Week_End','Year'] + list(yields.columns[1:])]
spx_weekly_check['Week_Return'] = spx_weekly_check.spx_Close/spx_weekly_check.spx_Open.shift(1) - 1
spx_weekly_check = spx_weekly_check[spx_weekly_check['Week_End'] == True]
spx_weekly_check['Up_Moves'] = (spx_weekly_check['Week_Return'] > 0.005)*1
spx_weekly_check['Down_Moves'] = (spx_weekly_check['Week_Return'] < -0.005)*1
spx_weekly_check['No_Moves'] = 1 - spx_weekly_check['Up_Moves'] - spx_weekly_check['Down_Moves']

weekly_counts = {'Ups':[],'Downs':[],'Flats':[]}
weekly_tally = spx_weekly_check[['Up_Moves','Down_Moves', 'No_Moves']]

for yr in spx_weekly_check.Year.drop_duplicates():
    weekly_counts['Ups'].append(weekly_tally[weekly_tally.index.year == yr].sum()['Up_Moves'])
    weekly_counts['Downs'].append(weekly_tally[weekly_tally.index.year == yr].sum()['Down_Moves'])
    weekly_counts['Flats'].append(weekly_tally[weekly_tally.index.year == yr].sum()['No_Moves'])
    
weekly_df = pd.DataFrame(weekly_counts, index = spx_weekly_check.Year.drop_duplicates())
weekly_df = weekly_df.drop([2002,2003,2004,2005,2006])
weekly_df_pct = pd.concat([(weekly_df['Ups']/weekly_df.sum(axis = 1)),
                           (weekly_df['Downs']/weekly_df.sum(axis = 1)),
                           (weekly_df['Flats']/weekly_df.sum(axis = 1))],axis = 1)
weekly_df_pct.columns = ['UpMove','DownMove','NoMove']
weekly_df_pct

,UpMove,DownMove,NoMove
Year,,,
1994,0.365385,0.288462,0.346154
1995,0.576923,0.173077,0.250000
1996,0.519231,0.326923,0.153846
1997,0.461538,0.365385,0.173077
1998,0.547170,0.320755,0.132075
1999,0.519231,0.365385,0.115385
2000,0.423077,0.538462,0.038462
2001,0.442308,0.442308,0.115385
2007,0.480769,0.346154,0.173077


In [6]:
spx_monthly_check = spx_monthly_check[['spx_Open','spx_Close','New_Month','Month_Expiry','Year'] + list(yields.columns[1:])]
spx_monthly_check['Month_Return'] = spx_monthly_check.spx_Close/spx_monthly_check.spx_Open.shift(1) - 1
spx_monthly_check = spx_monthly_check[spx_monthly_check['Month_Expiry'] == True]
spx_monthly_check['Up_Moves'] = (spx_monthly_check['Month_Return'] > 0.01)*1
spx_monthly_check['Down_Moves'] = (spx_monthly_check['Month_Return'] < -0.01)*1
spx_monthly_check['No_Moves'] = 1 - spx_monthly_check['Up_Moves'] - spx_monthly_check['Down_Moves']

monthly_counts = {'Ups':[],'Downs':[],'Flats':[]}
monthly_tally = spx_monthly_check[['Up_Moves','Down_Moves', 'No_Moves']]

for yr in spx_monthly_check.Year.drop_duplicates():
    monthly_counts['Ups'].append(monthly_tally[monthly_tally.index.year == yr].sum()['Up_Moves'])
    monthly_counts['Downs'].append(monthly_tally[monthly_tally.index.year == yr].sum()['Down_Moves'])
    monthly_counts['Flats'].append(monthly_tally[monthly_tally.index.year == yr].sum()['No_Moves'])
    
monthly_df = pd.DataFrame(monthly_counts, index = spx_monthly_check.Year.drop_duplicates())
monthly_df = monthly_df.drop([2002,2003,2004,2005,2006])
monthly_df_pct = pd.concat([(monthly_df['Ups']/monthly_df.sum(axis = 1)),
                           (monthly_df['Downs']/monthly_df.sum(axis = 1)),
                           (monthly_df['Flats']/monthly_df.sum(axis = 1))],axis = 1)
monthly_df_pct.columns = ['UpMove','DownMove','NoMove']
monthly_df_pct

,UpMove,DownMove,NoMove
Year,,,
1993,0.000000,0.000000,1.000000
1994,0.333333,0.250000,0.416667
1995,0.916667,0.000000,0.083333
1996,0.583333,0.166667,0.250000
1997,0.583333,0.333333,0.083333
1998,0.666667,0.250000,0.083333
1999,0.416667,0.166667,0.416667
2000,0.333333,0.583333,0.083333
2001,0.416667,0.416667,0.166667


In [7]:
monthly_df

,Downs,Flats,Ups
Year,,,
1993,0,1,0
1994,3,5,4
1995,0,1,11
1996,2,3,7
1997,4,1,7
1998,3,1,8
1999,2,5,5
2000,7,1,4
2001,5,2,5


In [150]:
contracts = spx_options.loc[[49,17]]
holding_days = 2
positions = [1, 1]
day_format = 'trading'
prem_price_use = 'Mid'

def ev_calc(contracts, positions, holding_days, price_stds = 5, day_format = 'trading', prem_price = 'Mid'):
    vix = curr_stock_data('^VIX')[-1]/100

    if day_format != 'trading':
        year = 365
    else:
        year = 252

    def kelly(odds, p, q):
        return (odds*p - q)/odds

    period_std = vix/np.sqrt(year/holding_days)

    returns = np.linspace(-price_stds*period_std,price_stds*period_std, 300)
    returns = pd.DataFrame({'Returns': returns,
                            'CDF': norm.cdf(returns, 0, period_std)}, index = range(len(returns)))

    returns['probs'] = round(returns['CDF'] - returns['CDF'].shift(1), 6).fillna(0)
    sims = []
    for idx, row in returns.iterrows():
        sims.append(position_sim(contracts, positions, 0, row.Returns, 0, holding_days, prem_price_use = prem_price))

    sim_df = pd.concat(sims,axis = 0)
    sim_df.index = range(len(returns))
    sim_df = sim_df.join(returns)[['PnL','probs','Percent Return']]

    sim_df['EV'] = sim_df.probs*sim_df.PnL
    expected_val = sim_df.EV.sum()
    expected_gain = sim_df[sim_df['EV'] >= 0].EV.sum()
    expected_loss = sim_df[sim_df['EV'] < 0].EV.sum()
    gain_prob = sim_df[sim_df['EV'] >= 0].probs.sum()
    loss_prob = sim_df[sim_df['EV'] < 0].probs.sum()

    out_df = pd.DataFrame({'EV': expected_val,
                           'Expected Win': expected_gain,
                           'Prob Win': gain_prob,
                           'Expected Loss': expected_loss,
                           'Prob Loss': loss_prob,
                           'Net Cost': (contracts.Mid*contracts.Pos).sum()*100,
                           'Kelly Fraction': kelly(expected_gain/((contracts.Mid*contracts.Pos).sum()*100), gain_prob, loss_prob)},
                          index = [0])
    
    return out_df

In [169]:
raw_options = all_options('NFLX', 60, 20, 0.03)
# raw_options.columns = ['Strike', 'Type', 'Expiry', 'DTE',
#                        'inTheMoney', 'Ask', 'Bid', 'Mid',
#                        'Last', 'IV', 'Underlying_Price']
# raw_options = raw_options.replace('c','call')
# raw_options = raw_options.replace('p','put')
raw_options

,Strike,Expiry,DTE,Type,IV,Underlying_Price,Last,Bid,Ask,Moneyness,Mid
0,360.0,2018-09-21,22,call,0.000010,368.49,17.86,0.0,0.0,0.023040,0.0
1,365.0,2018-09-21,22,call,0.000010,368.49,14.90,0.0,0.0,0.009471,0.0
2,370.0,2018-09-21,22,call,0.003916,368.49,12.50,0.0,0.0,0.004098,0.0
3,375.0,2018-09-21,22,call,0.015635,368.49,10.30,0.0,0.0,0.017667,0.0
4,360.0,2018-09-21,22,put,0.015635,368.49,8.82,0.0,0.0,0.023040,0.0
5,365.0,2018-09-21,22,put,0.007822,368.49,11.15,0.0,0.0,0.009471,0.0
6,370.0,2018-09-21,22,put,0.000010,368.49,13.55,0.0,0.0,0.004098,0.0
7,375.0,2018-09-21,22,put,0.000010,368.49,15.55,0.0,0.0,0.017667,0.0
8,357.5,2018-09-28,29,call,0.000010,368.49,20.60,0.0,0.0,0.029824,0.0
9,360.0,2018-09-28,29,call,0.000010,368.49,19.90,0.0,0.0,0.023040,0.0


In [170]:
import time
import warnings
warnings.filterwarnings('ignore')

start_time = time.time()

C = raw_options.copy()
greeks = all_greeks(C)

end_time = time.time()

print("Runtime: {}".format(end_time - start_time))

greeks

Runtime: 0.3609342575073242


,Strike,Expiry,DTE,Type,IV,Underlying_Price,Last,Bid,Ask,Moneyness,Mid,Rho,Vega,Gamma,Theta,Delta
0,360.0,2018-09-21,22,call,0.000010,368.49,17.86,0.0,0.0,0.023040,0.0,3.137566e-01,0.000000e+00,0.000000e+00,-1.900357e-02,1.000000e+00
1,365.0,2018-09-21,22,call,0.000010,368.49,14.90,0.0,0.0,0.009471,0.0,3.181143e-01,0.000000e+00,0.000000e+00,-1.926751e-02,1.000000e+00
2,370.0,2018-09-21,22,call,0.003916,368.49,12.50,0.0,0.0,0.004098,0.0,6.071234e-03,5.019858e-02,1.081314e-01,-6.761914e-04,1.888045e-02
3,375.0,2018-09-21,22,call,0.015635,368.49,10.30,0.0,0.0,0.017667,0.0,9.918922e-05,1.236725e-03,6.672769e-04,-3.634811e-05,3.086950e-04
4,357.5,2018-09-28,29,call,0.000010,368.49,20.60,0.0,0.0,0.029824,0.0,4.104960e-01,0.000000e+00,0.000000e+00,-1.886149e-02,1.000000e+00
5,360.0,2018-09-28,29,call,0.000010,368.49,19.90,0.0,0.0,0.023040,0.0,4.133666e-01,0.000000e+00,0.000000e+00,-1.899338e-02,1.000000e+00
6,362.5,2018-09-28,29,call,0.000010,368.49,17.95,0.0,0.0,0.016256,0.0,4.162372e-01,0.000000e+00,0.000000e+00,-1.912528e-02,1.000000e+00
7,365.0,2018-09-28,29,call,0.000010,368.49,16.50,0.0,0.0,0.009471,0.0,4.191078e-01,0.000000e+00,0.000000e+00,-1.925718e-02,1.000000e+00
8,367.5,2018-09-28,29,call,0.000010,368.49,15.80,0.0,0.0,0.002687,0.0,4.219784e-01,0.000000e+00,0.000000e+00,-1.938908e-02,1.000000e+00
9,370.0,2018-09-28,29,call,0.003916,368.49,14.00,0.0,0.0,0.004098,0.0,3.386303e-02,1.856688e-01,3.034059e-01,-2.421475e-03,7.990317e-02


In [171]:
#greeks[greeks.columns[4:]] = greeks[greeks.columns[4:]].apply(lambda col: np.round(col,6), axis = 1)
greeks[greeks['Mid'] != 0]

,Strike,Expiry,DTE,Type,IV,Underlying_Price,Last,Bid,Ask,Moneyness,Mid,Rho,Vega,Gamma,Theta,Delta
38,377.5,2018-09-28,29,put,0.475042,368.49,51.58,24.6,25.4,0.024451,25.0,-0.253912,0.497926,0.006708,-0.269897,-0.522148


In [172]:
position_df = greeks.loc[[24]]
position_df

,Strike,Expiry,DTE,Type,IV,Underlying_Price,Last,Bid,Ask,Moneyness,Mid,Rho,Vega,Gamma,Theta,Delta
24,370.0,2018-10-19,50,call,0.003916,368.49,22.8,0.0,0.0,0.004098,0.0,0.322073,0.647665,0.613852,-0.010334,0.441088


In [177]:
price_change = [-0.1, 0.1]
dte_change = [30, 50]
spacing = 50
holdings = [-1]

vol_change = [0, 0.1]
price_col = 'Last'
output = 'All'
shares = 100
skew = 'flat'

day_format = 'trading'
interest_rate = 0.0193
dividend_rate = 0
prem_price_use = 'Last'



In [178]:
start_time = time.time()


def position_sim(position_df, holdings, shares,
                 price_change, vol_change, dte_change, output = 'All',
                 skew = 'flat', prem_price_use = 'Mid', day_format = 'trading', 
                 interest_rate = 0.0193, dividend_rate = 0, vol_spacing = 2):

    position = position_df.copy().reset_index()[['Strike','Expiry','DTE','Type','IV','Underlying_Price',price_col]]
    position['Pos'] = holdings
    initial_cost = sum(position[price_col]*position['Pos'])*100 + shares*position['Underlying_Price'].values[0]
    
    price_changes = np.linspace(price_change[0], price_change[-1], spacing)
    dte_changes = np.linspace(dte_change[0], dte_change[-1], dte_change[-1] - dte_change[0] + 1)

    if vol_spacing <= 2:
        vol_changes = vol_change
    else:
        vol_changes = np.linspace(vol_change[0], vol_change[-1], vol_spacing)

    adj_dfs = []

    price_ax, dte_ax = np.meshgrid(price_changes,dte_changes)

    vol_adj_df = pd.DataFrame(np.array(np.meshgrid(price_changes,dte_changes)).reshape(2,-1).T)
    vol_adj_df.columns = ['ret_change', 'dte_change']

    for vol_change in vol_changes:
        # mesh_shape = np.meshgrid(price_changes,dte_changes)

        indi_sims = []
        for idx, row in position.iterrows():
            curr_sim = pd.DataFrame(index = range(len(vol_adj_df)))
            curr_sim['Strike'] = row.Strike
            curr_sim['DTE'] = row.DTE - vol_adj_df['dte_change']
            curr_sim[curr_sim['DTE'] < 0] = 0

            curr_sim['Type'] = row.Type
            curr_sim['IV'] = row.IV
            curr_sim['Underlying_Price'] = (1 + vol_adj_df[['ret_change']])*row.Underlying_Price
            curr_sim = price_sim(curr_sim, 0, vol_change, 0, output,
                                 skew, day_format, interest_rate, dividend_rate,
                                 prem_price_use)
            indi_sims.append(curr_sim)

        if len(holdings) < 2:
            try:
                adj_df = indi_sims[0].copy()
            except:
                break
            adj_df['Delta'] = adj_df['Delta'] + shares/100
            adj_df['PnL'] = position.head(1)['Pos'][0]*(adj_df['Simulated Prices'] - 
                                                        position.head(1)[price_col][0])*100 + shares*(adj_df['Underlying_Price'] - 
                                                                                                      position.head(1)['Underlying_Price'][0])
        else:
            adj_df = curr_sim[['Underlying_Price']]
            adj_df['Delta'] = 0
            adj_df['Gamma'] = 0
            adj_df['Vega'] = 0
            adj_df['Theta'] = 0
            adj_df['Rho'] = 0
            adj_df['PnL'] = 0
            for i, val in enumerate(holdings):
                adj_df['Delta'] = adj_df['Delta'] + val*indi_sims[i]['Delta']
                adj_df['Gamma'] = adj_df['Gamma'] + val*indi_sims[i]['Gamma']
                adj_df['Vega'] = adj_df['Vega'] + val*indi_sims[i]['Vega']
                adj_df['Theta'] = adj_df['Theta'] + val*indi_sims[i]['Theta']
                adj_df['Rho'] = adj_df['Rho'] + val*indi_sims[i]['Rho']
                adj_df['PnL'] = adj_df['PnL'] + val*indi_sims[i]['Simulated Prices']

            adj_df['PnL'] = (adj_df['PnL'] - sum(position[price_col]*position['Pos']))*100 - shares*(adj_df['Underlying_Price'] -
                                                                                                     position.head(1)['Underlying_Price'][0])
        if initial_cost < 0:
            adj_df['Percent Return'] = adj_df['PnL']/(-initial_cost)
        else:
            adj_df['Percent Return'] = adj_df['PnL']/initial_cost
        adj_df['Date'] = dt.datetime.today().date() + pd.to_timedelta(vol_adj_df['dte_change'] + 1, 'd')
        adj_dfs.append(adj_df)
    
    return (adj_dfs, price_ax, dte_ax)
# dte_ax = np.array(pd.to_datetime(adj_dfs[0].Date).dt.strftime('%m-%d-%Y')).reshape(dte_ax.shape)
    
end_time = time.time()
print("Runtime: {}".format(end_time - start_time))

Runtime: 0.0008463859558105469


In [179]:
# np.array(np.meshgrid(price_changes,dte_changes)).shape

# np.reshape(np.ravel(np.array(np.meshgrid(price_changes,dte_changes)).reshape(2,-1).T), 
#            np.array(np.meshgrid(price_changes,dte_changes)).shape)
adj_dfs, priceGrid, dteGrid = position_sim(position_df, holdings, shares,
                                         price_change, vol_change, dte_change)
test = position_df.copy()
test['DTE'] = 0
test['Underlying_Price'] = 377.5
black_scholes_merton(test, interest_rate = 0.0193, q = 0, year = 252)
adj_dfs[0]

,Strike,DTE,Type,IV,Underlying_Price,Rho,Vega,Theta,Gamma,Delta,Simulated Prices,PnL,Percent Return,Date
0,370.0,20.0,call,0.003916,331.641000,0.000000e+00,0.000000e+00,-0.000000e+00,0.000000e+00,1.000000,0.000000e+00,-1404.900000,-0.040640,2018-09-29
1,370.0,20.0,call,0.003916,333.145041,0.000000e+00,0.000000e+00,-0.000000e+00,0.000000e+00,1.000000,0.000000e+00,-1254.495918,-0.036290,2018-09-29
2,370.0,20.0,call,0.003916,334.649082,0.000000e+00,0.000000e+00,-0.000000e+00,0.000000e+00,1.000000,0.000000e+00,-1104.091837,-0.031939,2018-09-29
3,370.0,20.0,call,0.003916,336.153122,0.000000e+00,0.000000e+00,-0.000000e+00,0.000000e+00,1.000000,0.000000e+00,-953.687755,-0.027588,2018-09-29
4,370.0,20.0,call,0.003916,337.657163,0.000000e+00,0.000000e+00,-0.000000e+00,0.000000e+00,1.000000,0.000000e+00,-803.283673,-0.023237,2018-09-29
5,370.0,20.0,call,0.003916,339.161204,0.000000e+00,0.000000e+00,-0.000000e+00,0.000000e+00,1.000000,0.000000e+00,-652.879592,-0.018886,2018-09-29
6,370.0,20.0,call,0.003916,340.665245,0.000000e+00,0.000000e+00,-0.000000e+00,0.000000e+00,1.000000,0.000000e+00,-502.475510,-0.014535,2018-09-29
7,370.0,20.0,call,0.003916,342.169286,0.000000e+00,0.000000e+00,-0.000000e+00,0.000000e+00,1.000000,0.000000e+00,-352.071429,-0.010185,2018-09-29
8,370.0,20.0,call,0.003916,343.673327,0.000000e+00,0.000000e+00,-0.000000e+00,0.000000e+00,1.000000,0.000000e+00,-201.667347,-0.005834,2018-09-29
9,370.0,20.0,call,0.003916,345.177367,0.000000e+00,0.000000e+00,-0.000000e+00,0.000000e+00,1.000000,0.000000e+00,-51.263265,-0.001483,2018-09-29


In [180]:
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
v_changes = vol_change
display = 'PnL'
surface1 = go.Surface(x = (priceGrid + 1)*yahoo_data['Underlying_Price'].values[0], 
                      y = dteGrid, 
                      z = np.array(adj_dfs[0][display]).reshape(priceGrid.shape),
                      name = 'Vol Change of {}%'.format(round(v_changes[0]*100),2))

surface2 = go.Surface(x = (priceGrid + 1)*yahoo_data['Underlying_Price'].values[0], 
                      y = dteGrid, 
                      z = np.array(adj_dfs[1][display]).reshape(priceGrid.shape),
                      name = 'Vol Change of {}%'.format(round(v_changes[1]*100),2))

layout = go.Layout(
            title='{} Plot'.format(display),
            autosize=True,
            showlegend = False,
            scene=dict(
                aspectmode = 'manual',
                aspectratio = dict(x = 2,
                                   y = 2,
                                   z = 1),
                camera = dict(up = dict(x = 0,
                                        y = 0,
                                        z = 1),
                              center = dict(x = 0,
                                            y = 0,
                                            z = 0),
                              eye = dict(x = 1,
                                         y = 1,
                                         z = 0.5)),
                xaxis=dict(
                    title='Underlying Price',
                    gridcolor='rgb(255, 255, 255)',
                    zerolinecolor='rgb(255, 255, 255)',
                    showbackground=True,
                    backgroundcolor='rgb(230, 230,230)'
                ),
                yaxis=dict(
                    title='DTE Change',
                    gridcolor='rgb(255, 255, 255)',
                    zerolinecolor='rgb(255, 255, 255)',
                    showbackground=True,
                    backgroundcolor='rgb(230, 230,230)'
                ),
                zaxis=dict(
                    title = display,
                    gridcolor='rgb(255, 255, 255)',
                    zerolinecolor='rgb(255, 255, 255)',
                    showbackground=True,
                    backgroundcolor='rgb(230, 230,230)'
                )
            )
        )

data = [surface1, surface2]

iplot(dict(data = data,
           layout = layout))

In [84]:
sum(position_df['Mid']*holdings)*100

-186.49999999999983

In [85]:
position_df['Mid']

123     9.590
125    11.455
Name: Mid, dtype: float64

In [86]:
holdings

[1, -1]

In [87]:
9.59-11.455

-1.8650000000000002